# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creation of Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION={'class':'SimpleStrategy','replication_factor':1}    
    """
)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above

try:
    session.execute("USE sparkifydb")
except exception as e:
    Print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## for this requirement sessionId, SessionItem will make unique required so they should consider for primary keys
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS songs_session (sessionId int, itemInSession int, artist text, songTitle text, songLength decimal, PRIMARY KEY(sessionId, itemInSession))
    """)
except exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_session (sessionId, itemInSession, artist, songTitle, songLength)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query,(int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
#selecting artist songtitle and length of sessionid 338 and sessionitem 4
select_query='select artist, songTitle, songLength from songs_session where sessionId=338 and itemInSession=4'
output=session.execute(select_query)

for row in output:
    print(row)

Row(artist='Faithless', songtitle='Music Matters (Mark Knight Dub)', songlength=Decimal('495.3073'))


In [14]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## for this requirement userId and sessionId are composite keys, SessionItem will be clustering key since we need to sort the result.

try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS users_songs (userId int, sessionId int, itemInSession int, artist text, songTitle text, firstName text, lastName text,  
    PRIMARY KEY((userId, sessionId), itemInSession))
    WITH CLUSTERING ORDER BY (itemInSession DESC);
    """)
except exception as e:
    print(e)
  


                    

In [15]:
  
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_songs (userId, sessionId,itemInSession, artist, songTitle, firstName, lastName )"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query,(int(line[10]), int(line[8]),int(line[3]), line[0], line[9], line[1],line[4]))
        except Exception as e:
            print(e)


In [16]:
# Selecting artist and user name who's id was 10 and sessionid was 182 
select_query='SELECT artist, songTitle, firstName, lastName FROM users_songs WHERE userId = 10 AND sessionId = 182'
output=session.execute(select_query)

for row in output:
    print(row)

Row(artist='Lonnie Gordon', songtitle='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', songtitle='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', songtitle='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Down To The Bone', songtitle="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')


  ### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [18]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## for this requirement songtile is primary key since we need user who have listened to particular song

try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS music_history (songTitle text, userId int, sessionId int, firstName text, lastName text, PRIMARY KEY(songTitle, userId))
    """)
except exception as e:
    print(e)


                    

In [20]:
  
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_history (songTitle, userId, sessionId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query,(line[9],int(line[10]), int(line[8]), line[1],line[4]))
        except Exception as e:
            print(e)


In [21]:
## selecting firstname and lastname of users who listened to song 'All Hands Against His OWn'

select_query="SELECT firstName, lastName FROM music_history WHERE songTitle ='All Hands Against His Own'"
output=session.execute(select_query)

for row in output:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [22]:
session.execute('drop table songs_session')
session.execute('drop table users_songs')
session.execute('drop table music_history')

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()